In [ ]:
import sys
from importlib import reload

reload(sys.modules['utils']);
reload(sys.modules['trainHelper']);
reload(sys.modules['lossFunction']);
reload(sys.modules['model.RNN']);

In [ ]:


import trainHelper
import utils
import numpy
from torch.utils.data import DataLoader

import torch
from torch import nn, Tensor
from torch.nn import functional as F

from model.AutoEncoder import AutoEncoder
from model.Linear import Linear

import model.RNN as rnn

import os.path 
import glob

from lossFunction import CoordsToDMLoss, ReconLoss

In [ ]:
sample_space = (1000, 1)
ss, N, d = 1600, 10, 2

In [ ]:
try:
    euclidean_data1 = utils.load_variable('data/euclidean_data1.pkl')
    euclidean_data2 = utils.load_variable('data/euclidean_data2.pkl')
    
    rand_data1 = utils.load_variable('data/rand_data1.pkl')
    rand_data2 = utils.load_variable('data/rand_data2.pkl')
    
    if euclidean_data1.size() != (ss, 1, N, N):
        raise Exception("Previous data not match requirement !")

except:
    
    print("Updated data for requirement !")

    euclidean_data1 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    euclidean_data2 = utils.generate_euclidean_DM(
        N=N, d=d,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data1 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    rand_data2 = utils.generate_rand_DM(
        N=N,
        sample_size=ss,
        sample_space=sample_space, isInt=True)

    utils.dump_variable(euclidean_data1, 'data/euclidean_data1.pkl')
    utils.dump_variable(euclidean_data2, 'data/euclidean_data2.pkl')

    utils.dump_variable(rand_data1, 'data/rand_data1.pkl')
    utils.dump_variable(rand_data2, 'data/rand_data2.pkl')

In [ ]:
data = torch.stack([
            # euclidean_data1,
            # euclidean_data2,
            rand_data1,
            rand_data2,
        ]) 

data = data.view(ss * 2, 1, N, N)

data = utils.minmax_norm(data, dmin=0)[0]
data = torch.tensor(data.data, requires_grad=True)

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

data[0, 0, :3, :3].data

In [ ]:
def train(helper):
    for i in range(10):

        EPOCH = 100
            
        print("Training ", helper.id)

        helper.train(dlr, EPOCH, print_on_each=49)
        helper.backup()
            
        min_lr = helper.records['lr'].min()
            
        print("Time used for the training: ", 
                    helper.records['train_time'].sum(), "s", 
                "| Last lr: ", min_lr)

        if 1e-7 > min_lr and helper.records['lr'][helper.records['lr'] == min_lr].count() >= EPOCH:
            print("Reach Low learning Rate !")
            break

In [ ]:
def preprocess(x):
    N = x.size()[-1]
    x = x.view(-1, N, N)
    return torch.tensor(x.data, requires_grad=True)

for neuron in range(16, 97, 16):

    for i in range(1, 4):

        model_id = "Coord_bRNN_" + str(i)+ "_" + str(neuron) + "_M_MSE"

        model = rnn.bRNN(N, 2, [N, *[neuron for j in range(i)], int(neuron / 2)], num_rnn_layers=1)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        lossFun = CoordsToDMLoss(N, 2, lossFun=nn.MSELoss(reduction='sum'))

        helper = trainHelper.TrainHelper(id=model_id, 
            model=model, optimizer=optimizer, preprocess=preprocess, lossFun=lossFun)

        train(helper)

In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in range(16, 97, 16):

    for i in range(0, 6):

        model_id = "Coord_Linear_" + str(i)+ "_" + str(neuron) + "_D_MSE"

        in_dim = int((N * N - N) / 2)
        out_dim = N * 2

        model = Linear([in_dim, *[neuron, int(neuron / 2) for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = CoordsToDMLoss(N, 2, lossFun=nn.MSELoss(reduction='sum'))
        
        helper = trainHelper.TrainHelper(id=model_id,
            model=model, optimizer=optimizer, preprocess=preprocess, lossFun=lossFun)
        
        train(helper)

In [ ]:

def preprocess(x):
    return torch.tensor(x.data, requires_grad=True)
    
for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "Coord_Linear_" + str(i)+ "_" + str(neuron) + "_M_MSE"

        in_dim = N
        out_dim = 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        
        lossFun = CoordsToDMLoss(N, 2)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, optimizer=optimizer, preprocess=preprocess, lossFun=lossFun)
        
        train(helper)
        

In [ ]:
def preprocess(x):
    batch = x.size()[0]
    N = x.size()[-1]
    ex = utils.eignmatrix(x.view(batch, N, N)).view(batch, 1, N, N * 2)
    return torch.tensor(ex.data, requires_grad=True)

for neuron in [32]:

    for i in range(6, 12):

        model_id = "Linear_" + str(i)+ "_" + str(neuron) + "_E_MSE"

        in_dim = N * 2
        out_dim = 2

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = CoordsToDMLoss(N, 2)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, optimizer=optimizer, preprocess=preprocess, lossFun=lossFun)
        
       train(helper)

In [ ]:
data = torch.stack([
            euclidean_data1,
            euclidean_data2,
            # rand_data1,
            # rand_data2,
        ])
        
data = data.view(ss * 2, 1, N, N)

data = utils.minmax_norm(data, dmin=0)[0]
data = torch.tensor(data.data, requires_grad=True)

batch = 32
dlr = DataLoader(data, batch_size=batch, shuffle=True)

In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "Linear_AE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = int((N * N - N) /2) 

        model = Linear([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = ReconLoss()

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break
                
            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())

In [ ]:

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "AE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = AutoEncoder([in_dim, *[neuron for j in range(i)], out_dim],
                 activation=nn.LeakyReLU, 
                 final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        
        lossFun = ReconLoss()

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        
        for i in range(5):

            EPOCH = 300
            
            print("Training ", helper.id)

            try: 
                helper.train(dlr, EPOCH, print_on_each=100)
                helper.backup()
            except:
                break

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s", "| Last lr: ", helper.records['lr'].min())

In [ ]:
raise Exception("End")

def MSES(model_dm, target_dm):
    
    model_dm = utils.minmax_norm(model_dm)[0]
    return nn.functional.binary_cross_entropy(model_dm, target_dm, reduction='sum')

def preprocess(x):
    return torch.tensor(
        utils.vectorize_distance_from_DM(x).data, requires_grad=True)

for neuron in [32, 64, 96]:

    for i in range(2, 6):

        model_id = "VAE_" + str(i)+ "_" + str(neuron) + "_distance_LReLU"

        in_dim = int((N * N - N) /2) 
        out_dim = N * 2

        model = VAE([in_dim, *[neuron for j in range(i)], out_dim],
                    activation=nn.LeakyReLU,
                    final_activation=None)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        
        lossFun = lossF.VAELoss(lossFun=MSES)

        helper = trainHelper.TrainHelper(id=model_id,
            model=model, 
            optimizer=optimizer, 
            preprocess=preprocess,
            lossFun=lossFun, lr_factor=0.1)
        
        for i in range(3):

            EPOCH = 300
            
            print("Training ", helper.id)

            helper.train(dlr, EPOCH, print_on_each=100)
            helper.backup()

            print("Time used for the training: ", 
                            helper.records['train_time'].sum(), "s")    


In [ ]:
l = torch.triu(data[:3])

In [ ]:
l